In [1]:
import pandas as pd
import numpy as np
import matplotlib 
from matplotlib.pyplot import plot as pyplt

In [2]:
import json
import os

# Opening JSON file
f = open('/project_data/data_asset/Predict_Envs.json',)
data = json.load(f)
f.close()

os.environ['APM_ID'] = data['APM_ID']
os.environ['APM_API_BASEURL'] = data['APM_API_BASEURL']
os.environ['APM_API_KEY'] = data['APM_API_KEY']

# README: Change to true if Health/Predict are deployed on different cluster from this CP4D environment
use_external = False
if use_external:
    import os
    os.environ['USER_PROVIDED_HEALTH_URL'] = data['USER_PROVIDED_HEALTH_URL']
    os.environ['USER_PROVIDED_DB_CONNECTION_STRING'] = data['USER_PROVIDED_DB_CONNECTION_STRING']
    os.environ['USER_PROVIDED_URL'] = data['USER_PROVIDED_URL']

In [3]:
import os
os.environ['TRUST_PREDICT']= os.getenv('APM_API_BASEURL')[8:]

In [4]:
!pip install --trusted-host ${TRUST_PREDICT} -U ${APM_API_BASEURL}/ibm/pmi/service/rest/ds/${APM_ID}/${APM_API_KEY}/lib/download?filename=pmlib

     |████████████████████████████████| 189.3 MB 53.0 MB/s eta 0:00:01   |██████████▍                     | 61.8 MB 21.7 MB/s eta 0:00:06  | 96.8 MB 21.7 MB/s eta 0:00:05��█████████████▋              | 104.5 MB 21.7 MB/s eta 0:00:04███▌        | 139.2 MB 65.2 MB/s eta 0:00:01�██████████▉        | 140.7 MB 65.2 MB/s eta 0:00:01     |█████████████████████████████▎  | 173.0 MB 53.0 MB/s eta 0:00:01
Processing //tmp/1000690000/pip-req-build-rddsc8nd/dslib/srom-1.4.0rc7.1.0-cp38-cp38-linux_x86_64.whl
Processing //tmp/1000690000/pip-req-build-rddsc8nd/dslib/dqlearn-0.4.2.1.0-cp38-cp38-linux_x86_64.whl
Processing //tmp/1000690000/pip-req-build-rddsc8nd/dslib/aixclient.zip


dqlearn is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
srom is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
  Created wheel for aixclient: filename=aixclient-1.2.2-py3-none-any.whl size=5088 sha256=7b5f1f7b95c12fb41f626966a0c01f8cf9be9d48f4ebfacb0e5d0a7e6e1313f7
  Stored in directory: /tmp/1000690000/pip-ephem-wheel-cache-ess9gfd3/wheels/0c/11/ac/e0962c499da6b5e7d7743762ce729df64e7153a549214cd243
Successfully built aixclient
  Attempting uninstall: aixclient
    Found existing installation: aixclient 1.2.2
    Uninstalling aixclient-1.2.2:
      Successfully uninstalled aixclient-1.2.2


In [5]:
import srom
srom.__version__

'1.4.0rc7'

In [6]:
import dqlearn
dqlearn.__version__

'0.4.2'

In [7]:
import pmlib
from pmlib import api
#pmlib.set_log_level('CRITICAL')
api.init_environ()

2022-07-19T15:43:42.936 tensorflow.<module> DEBUG Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2022-07-19T15:43:43.860 pmlib.api.init_environ INFO APM_ID=d17a8d18, APM_API_BASEURL=https://ivt11rel87-predict-api.mas-ivt11rel87-predict.svc, APM_API_KEY=********
2022-07-19T15:43:43.863 pmlib.util.api_request INFO method=get, url=https://ivt11rel87-predict-api.mas-ivt11rel87-predict.svc/ibm/pmi/service/rest/ds/tenant?instanceId=d17a8d18, headers={'apmapitoken': '********'}, timeout=300, ssl_verify=False, json=None, session=None, kwargs={}
2022-07-19T15:43:46.276 pmlib.util.api_request INFO resp.status_code=200, method=get, url=https://ivt11rel87-predict-api.mas-ivt11rel87-predict.svc/ibm/pmi/service/rest/ds/tenant?instanceId=d17a8d18
2022-07-19T15:43:46.279 pmlib.api.init_environ DEBUG resp={
    "as_apikey": "********",
    "as_apitoken": "********",
    "as_id": null,
    "as_url": "https://main.api.monitor

('d17a8d18',
 'https://ivt11rel87-predict-api.mas-ivt11rel87-predict.svc',
 'hhgjlmhelthqhacki6s820cdb2qotp3o0pgvaob8')

In [8]:
#use the following flags to suppress the pointless pyspark logs that fill up space
import logging
logging.getLogger('pyspark').setLevel(logging.ERROR)
logging.getLogger('py4j').setLevel(logging.ERROR)
logging.getLogger('py4j.java_gateway').setLevel(logging.ERROR)
logging.getLogger(' __PROJECT_LIB__').setLevel(logging.ERROR)

In [9]:
import json
from project_lib import Project
project = Project.access()

2022-07-19 15:43:46,628 - __PROJECT_LIB__ - INFO - GET https://internal-nginx-svc.cpd-services.svc.cluster.local:12443/v2/projects/6a4bd1de-3f17-4333-8d46-64628ba44971


2022-07-19T15:43:46.628 __PROJECT_LIB__._get INFO GET https://internal-nginx-svc.cpd-services.svc.cluster.local:12443/v2/projects/6a4bd1de-3f17-4333-8d46-64628ba44971


In [11]:
sensor_asset_id_column_name = 'asset_id'
sensor_date_column_name = 'timestamp'

In [13]:
sensor_data_full_df = pd.read_csv('/project_data/data_asset/kaggle_pump_resampled_reindexed.csv')
#print(sensor_data_df.columns.tolist())
if 'Unnamed: 0' in sensor_data_full_df.columns.tolist():
    sensor_data_full_df.drop('Unnamed: 0', axis = 1, inplace = True)
if 'index' in sensor_data_full_df.columns.tolist():
    sensor_data_full_df.drop('index', axis = 1, inplace = True)
sensor_data_full_df[sensor_asset_id_column_name] = 'pump-123'
sensor_data_full_df = sensor_data_full_df.drop_duplicates(subset=[sensor_asset_id_column_name, sensor_date_column_name])
sensor_data_full_df[sensor_date_column_name] = pd.to_datetime(sensor_data_full_df[sensor_date_column_name])
print('Machine Status values = ', sensor_data_full_df['machine_status'].value_counts())
print(sensor_data_full_df.shape)
print(sensor_data_full_df.isna().sum())
display(sensor_data_full_df.head())
sensor_data_full_df.tail()

Machine Status values =  NORMAL        1138
RECOVERING      79
BROKEN           7
Name: machine_status, dtype: int64
(1224, 55)
timestamp            0
sensor_00           52
sensor_01            0
sensor_02            0
sensor_03            0
sensor_04            0
sensor_05            0
sensor_06           23
sensor_07           27
sensor_08           25
sensor_09           22
sensor_10            0
sensor_11            0
sensor_12            0
sensor_13            0
sensor_14            0
sensor_15         1224
sensor_16            0
sensor_17            0
sensor_18            0
sensor_19            0
sensor_20            0
sensor_21            0
sensor_22            0
sensor_23            0
sensor_24            0
sensor_25            0
sensor_26            0
sensor_27            0
sensor_28            0
sensor_29            0
sensor_30            0
sensor_31            0
sensor_32            0
sensor_33            0
sensor_34            0
sensor_35            0
sensor_36            

timestamp  sensor_00  sensor_01  sensor_02  sensor_03   sensor_04  \
0 2019-11-25   2.454124  48.521651  53.086659  45.917967  634.261195   
1 2019-11-26   2.454091  48.067611  52.885319  45.239437  634.028005   
2 2019-11-27   2.454162  46.882473  52.547741  44.823735  633.935177   
3 2019-11-28   2.454222  46.601319  52.434171  45.273677  634.954348   
4 2019-11-29   2.453987  47.860242  52.780427  46.145350  634.696986   

   sensor_05  sensor_06  sensor_07  sensor_08  ...  sensor_44  sensor_45  \
0  75.106401  13.517834  16.159856  15.694364  ...  41.407857  49.429334   
1  72.457242  13.597486  16.157967  15.696575  ...  53.991448  61.837706   
2  74.061776  13.513292  16.160580  15.666554  ...  55.351402  55.592528   
3  73.439419  13.344946  16.167010  15.678169  ...  54.710005  50.724987   
4  75.166017  13.351055  16.161303  15.673628  ...  59.770448  52.858153   

   sensor_46  sensor_47   sensor_48   sensor_49   sensor_50   sensor_51  \
0  46.482768  39.031957  183.920076   75.800541  231.928371  207.921807   
1  51.240998  38.736175  200.657471  103.338798  245.225696  216.391781   
2  42.742091  39.072144  214.248972  108.018260  241.538062  210.863552   
3  44.700038  42.306456  200.869662   96.164481  242.978392  212.594844   
4  48.866705  42.751736  202.019033  101.538388  231.573112  220.992476   

   asset_id  machine_status  
0  pump-123          NORMAL  
1  pump-123          NORMAL  
2  pump-123          NORMAL  
3  pump-123          NORMAL  
4  pump-123          NORMAL  

[5 rows x 55 columns]

timestamp  sensor_00  sensor_01  sensor_02  sensor_03   sensor_04  \
1219 2023-03-28   2.402791  51.593362  51.257954  44.655670  627.556543   
1220 2023-03-29   2.420822  50.335646  52.329763  45.108264  630.655406   
1221 2023-03-30   2.402808  50.565199  52.166761  44.605274  632.023558   
1222 2023-03-31   2.402592  49.398244  51.008070  43.506011  631.933956   
1223 2023-04-01   2.402649  47.933544  50.588349  43.128857  631.943893   

      sensor_05  sensor_06  sensor_07  sensor_08  ...  sensor_44  sensor_45  \
1219  77.754073  14.987380  16.526290  15.708309  ...  76.284401  65.266525   
1220  73.575111  15.106818  16.932628  15.960086  ...  88.118893  71.498842   
1221  75.681375  15.112043  17.167246  16.185417  ...  71.024626  66.642554   
1222  72.777791  15.116304  16.803865  15.803025  ...  67.585726  55.720666   
1223  70.282040  15.110556  16.658832  15.674793  ...  62.618955  56.540959   

      sensor_46  sensor_47   sensor_48   sensor_49  sensor_50   sensor_51  \
1219  59.722222  44.047389  279.935054  131.312694        NaN  234.799381   
1220  65.043081  50.483861  315.470678  168.688913        NaN  245.492544   
1221  57.101980  45.534336  241.833846   99.614199        NaN  192.677789   
1222  54.664613  41.153353  260.241748  115.474271        NaN  220.535154   
1223  55.135995  44.523211  287.753986  115.928823        NaN  219.936341   

      asset_id  machine_status  
1219  pump-123          NORMAL  
1220  pump-123          NORMAL  
1221  pump-123          NORMAL  
1222  pump-123          NORMAL  
1223  pump-123          NORMAL  

[5 rows x 55 columns]

In [14]:
# From the output above, it is clear that sensor_15 is missing values in all rows. So let us remove that variable
print('Shape of the data frame BEFORE dropping sensor_15 = ', sensor_data_full_df.shape)
sensor_data_full_df.drop('sensor_15', axis = 1, inplace = True)
print('Shape of the data frame AFTER dropping sensor_15 = ', sensor_data_full_df.shape)

Shape of the data frame BEFORE dropping sensor_15 =  (1224, 55)
Shape of the data frame AFTER dropping sensor_15 =  (1224, 54)


In [15]:
sensor_measurements = sorted(list(set(sensor_data_full_df.columns.tolist()) - set(['machine_status', 'target_label', sensor_asset_id_column_name, sensor_date_column_name])))
print(sensor_measurements)

['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25', 'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30', 'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35', 'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 'sensor_40', 'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44', 'sensor_45', 'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49', 'sensor_50', 'sensor_51']


Perform Missing Value Analysis

In [16]:
from dqlearn.missing.missing_pattern_plot import missing_pattern_plots
from dqlearn.missing.missing_pattern import check_missing_properties

missing_threshold_dict = {}
missing_threshold_dict = {var:0.002 for var in sensor_measurements}

sensor_report = check_missing_properties(df=sensor_data_full_df, data_cols=sensor_measurements, missing_threshold_dict=missing_threshold_dict, default_missing_threshold=0.005,
                                         verify_df=True)
#print(sensor_report()['results']['explanation'])
#print(sensor_report()['results']['details']['feature_missing_properties'])
#print(sensor_report()['results']['explanation'])

sensor_report_dict = sensor_report()
sensor_report_json = json.dumps(sensor_report_dict, indent=4)

print(sensor_report_json)

sensor_list_dict = sensor_report_dict['results']['details']['feature_missing_properties']
sensor_missing_properties_df = pd.DataFrame(sensor_list_dict)
sensor_missing_properties_df = sensor_missing_properties_df.transpose()
#sensor_missing_properties_df = sensor_missing_properties_df[missing_properties].transpose()
pd.set_option("display.max_colwidth", -1)  # Ensure all the contents diaplayed
sensor_missing_properties_df.head(sensor_missing_properties_df.shape[0])

2022-07-19T15:47:35.947 numexpr.utils._init_num_threads INFO Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
missing_gap_dict OrderedDict([(1, 49), (38, 1), (564, 1)])
missing_gap_dict OrderedDict([(1, 25), (335, 1)])
missing_gap_dict OrderedDict([(1, 23), (584, 1)])
{
    "results": {
        "title": "Check the missing value properties for features",
        "score": null,
        "explanation": "There are(is) 7 out of 51 features(s) have/has missing value(s)!\nThere are(is) 7 out of 51 features(s) have/has missing ratio(s) higher than the threshold(s)!",
        "recommendations": null,
        "details": {
            "feature_list": [
                "sensor_00",
                "sensor_01",
                "sensor_02",
                "sensor_03",
                "sensor_04",
                "sensor_05",
                "sensor_06",
                "sensor_07",
                "sensor_08",
                "sensor_09",
             

sample_size num_missing contain_missing missing_rate  \
sensor_00  1224        52          True            0.042         
sensor_01  1224        0           False           0.0           
sensor_02  1224        0           False           0.0           
sensor_03  1224        0           False           0.0           
sensor_04  1224        0           False           0.0           
sensor_05  1224        0           False           0.0           
sensor_06  1224        23          True            0.019         
sensor_07  1224        27          True            0.022         
sensor_08  1224        25          True            0.02          
sensor_09  1224        22          True            0.018         
sensor_10  1224        0           False           0.0           
sensor_11  1224        0           False           0.0           
sensor_12  1224        0           False           0.0           
sensor_13  1224        0           False           0.0           
sensor_14  1224        0           False           0.0           
sensor_16  1224        0           False           0.0           
sensor_17  1224        0           False           0.0           
sensor_18  1224        0           False           0.0           
sensor_19  1224        0           False           0.0           
sensor_20  1224        0           False           0.0           
sensor_21  1224        0           False           0.0           
sensor_22  1224        0           False           0.0           
sensor_23  1224        0           False           0.0           
sensor_24  1224        0           False           0.0           
sensor_25  1224        0           False           0.0           
sensor_26  1224        0           False           0.0           
sensor_27  1224        0           False           0.0           
sensor_28  1224        0           False           0.0           
sensor_29  1224        0           False           0.0           
sensor_30  1224        0           False           0.0           
sensor_31  1224        0           False           0.0           
sensor_32  1224        0           False           0.0           
sensor_33  1224        0           False           0.0           
sensor_34  1224        0           False           0.0           
sensor_35  1224        0           False           0.0           
sensor_36  1224        0           False           0.0           
sensor_37  1224        0           False           0.0           
sensor_38  1224        0           False           0.0           
sensor_39  1224        0           False           0.0           
sensor_40  1224        0           False           0.0           
sensor_41  1224        0           False           0.0           
sensor_42  1224        0           False           0.0           
sensor_43  1224        0           False           0.0           
sensor_44  1224        0           False           0.0           
sensor_45  1224        0           False           0.0           
sensor_46  1224        0           False           0.0           
sensor_47  1224        0           False           0.0           
sensor_48  1224        0           False           0.0           
sensor_49  1224        0           False           0.0           
sensor_50  1224        427         True            0.349         
sensor_51  1224        84          True            0.069         

          missing_rate_lower_threshold missing_rate_threshold  \
sensor_00  False                        0.002                   
sensor_01  True                         0.002                   
sensor_02  True                         0.002                   
sensor_03  True                         0.002                   
sensor_04  True                         0.002                   
sensor_05  True                         0.002                   
sensor_06  False                        0.002                   
sensor_07  False                        0.002            

Auto Impute the values For the purposes of this exercise you can choose to ignore MCAR flag.

In [17]:
from srom.auto.auto_imputation import AutoImputation

scoring_strategy = "neg_mean_absolute_error"  # any sklearn defined regression metric is fine

# possible values include ImputationKFold, MARImputationKFold, and MNARImputationKFold, ImputationKFold, 
# Other two candidate kfold types for time series are: a) TsIIDConsecutiveKFold, b) TsVariableConsecutiveKFold
cv_type = "ImputationKFold"
imputation_type = "timeseries" # either iid or timeseries
cv = 5
total_execution_time = 10

sensor_data_imputation_ts = AutoImputation(
    level='default', 
    save_prefix='auto_imputation_timeseries', 
    execution_platform='spark_node_random_search', 
    cv_type=cv_type,  
    cv=cv,
    scoring=scoring_strategy, 
    stages=None, 
    execution_time_per_pipeline=2, 
    num_options_per_pipeline_for_random_search=10,
    num_option_per_pipeline_for_intelligent_search=30, 
    total_execution_time=total_execution_time, 
    param_grid=None, 
    missing_vals=0.1, 
    random_state=42,
    imputation_type=imputation_type)


sensor_X = sensor_data_full_df[sensor_measurements].values

%time sensor_data_imputation_ts.automate(sensor_X, sensor_X)

sensor_data_imputation_ts.fit(sensor_X, sensor_X)
sensor_data_imputed_ts = sensor_data_imputation_ts.transform(sensor_X)

sensor_data_full_imputed_df = pd.DataFrame(sensor_data_imputed_ts, columns=sensor_measurements)
sensor_data_full_imputed_df[[sensor_asset_id_column_name, sensor_date_column_name,'machine_status']] = sensor_data_full_df[[sensor_asset_id_column_name, sensor_date_column_name,'machine_status']]
print(sensor_data_full_imputed_df.isna().sum())
sensor_data_full_imputed_df[[sensor_asset_id_column_name, sensor_date_column_name, 'target_label', 'machine_status']] = \
sensor_data_full_df[[sensor_asset_id_column_name, sensor_date_column_name, 'target_label', 'machine_status']]
sensor_data_full_imputed_df = sensor_data_full_imputed_df[[sensor_asset_id_column_name, sensor_date_column_name]+sensor_measurements+['machine_status','target_label']]
sensor_data_full_imputed_df.head()

2022-07-19T15:48:16.250 srom.utils.pipeline_utils.<module> INFO No grid parameter matched for the following stages: cubicimputer, akimaimputer, barycentricimputer, splineImputer, quadraticimputer, linearimputer
2022-07-19T15:48:34.870 srom.pipeline.execution_types.srom_spark_search.<module> DEBUG ***** score_results in srom_spark_search is [(0, ({}, -0.6646053580167194, 0.019514402134049137, 0.00039411749306407425, 0.011709953993212598)), (1, ({}, -0.9221869830776577, 0.03132805043831078, 0.0004935287300463025, 0.5317739174151419)), (2, ({}, -0.7968328787937654, 0.019972262223949523, 0.0002505177090678408, 0.008218399099949864)), (3, ({}, -0.7690238264879509, 0.019702447231469248, 0.00017388582084143392, 0.04702834044521912)), (4, ({}, -0.6771062785195265, 0.01936839248765523, 0.00029307425761035806, 0.029524896299974356)), (5, ({'polynomialimputer__order': 3}, -0.7968328787937654, 0.019972262223949523, 0.00021667773243108125, 0.02418716012823859)), (6, ({}, -0.6882350856572198, 0.0198

KeyError: "['target_label'] not in index"

Export the dataset. Make sure you use the same name in the Data Loader notebook if you choose to load this imputed dataset into Monitor

In [19]:
sensor_data_full_imputed_df.to_csv('kaggle_pump_reindexed_imputed.csv', index = False)